# IndexFlatL2演示

## 目录
- faiss简介
- 演示
    - 构造测试训练数据
    - 训练
    - 用k-近邻搜索
    - 完整代码
- 参考

# faiss简介

- 由Facebook AI Research开发
- 用C++编写
- 提供Python接口
- 一些核心算法提供了GPU实现
- 是一个用于相似性搜索和密集向量聚类的高性能库，支持十亿级别向量的搜索，是目前最为成熟的相似性搜索库
- 为高维向量提供精确或近似最近邻搜索的功能
- 无监督学习
- 包含多种搜索任意大小向量集的算法，以及用于算法评估和参数调整的支持代码。向量集大小由内存决定
- 它假设实例被表示为向量，并用整数标识，可以用L2距离或点积进行比较。与查询向量相似的向量，是那些与该具有最低L2距离或最高点积的向量。它还支持余弦相似性，因为这是归一化向量上的点积。
- 大多数方法，如基于二进制向量和紧凑量化码的方法，仅使用向量的压缩表示，而不需要保留原始向量。这通常以不太精确的搜索为代价，这些方法可以在单个服务器的主内存中进行数十亿向量集的搜索。
- GPU实现支持单或多GPU

# IndexFlatL2演示

In [1]:
import numpy as np
import faiss
import time

### 构造测试训练数据

下面构造如下向量集合：
1. 10万个向量集合
2. 每个向量64个维度
3. 进行1万次相似性搜索

In [2]:
d = 64                           
nb = 100000                       
nq = 10000                       
np.random.seed(1234)                              # 对生成的数据，每次都一样，可预见，若无此句，每次都不一样
xb = np.random.random((nb, d)).astype('float32')  # 向量集合，矩阵nb*d
xb[:, 0] += np.arange(nb) / 1000                  # 修正向量集合第一列，每项加 0...nb/1000 小数
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000 
print(xq)

[[ 0.81432974  0.7409969   0.8915324  ...  0.72459674  0.893881
   0.6574571 ]
 [ 0.5844774   0.797842    0.74140453 ...  0.6768835   0.05907924
   0.6396156 ]
 [ 0.75040764  0.02659794  0.5495097  ...  0.69562465  0.16268532
   0.76653737]
 ...
 [10.96773     0.05037309  0.7342035  ...  0.89510185  0.6490696
   0.86151606]
 [10.831193    0.70606154  0.1922274  ...  0.8026039   0.6854174
   0.60209423]
 [10.078484    0.39106598  0.01359335 ...  0.63193923  0.12561724
   0.78384215]]


> 说明：
>
> - Faiss处理固定维数d的向量集合（矩阵），d通常为几十到几百。
>
> - `xb`为`[100000,64]`的训练数据，`xq`为`[10000,64]`的查询数据。
>
> - 只使用32位浮点矩阵。
>
> - 行主存储，即向量编号为`i`的第`j`个分量存储在矩阵的第`i`行第`j`列中。
> 
> 例如，如下向量集合为5个数据，每个数据为2维度：
> ```
> [[0.19151945 0.62210876]
>  [0.43872774 0.7853586 ]
>  [0.7819758  0.2725926 ]
>  [0.27946424 0.8018722 ]
>  [0.96213937 0.87593263]]
> ```

`add`方法一般添加训练时的样本，`search`方法是查找相似向量。

### 训练

- faiss创建索引从而对向量预处理，提高查询效率。
- faiss最简单的暴力检索L2距离：`IndexFlatL2`。`IndexFlatL2`是评价基线。
- 创建索引时，必须指定数据集向量维度`d`。 

In [3]:
index = faiss.IndexFlatL2(d)
print('is_trained',index.is_trained)
index.add(xb)
print('ntotal', index.ntotal)

is_trained True
ntotal 100000


In [4]:
index.reconstruct(0)

array([0.19151945, 0.62210876, 0.43772775, 0.7853586 , 0.77997583,
       0.2725926 , 0.27646425, 0.8018722 , 0.95813936, 0.87593263,
       0.35781726, 0.5009951 , 0.6834629 , 0.71270204, 0.37025076,
       0.5611962 , 0.50308317, 0.01376845, 0.7728266 , 0.8826412 ,
       0.364886  , 0.6153962 , 0.07538124, 0.368824  , 0.9331401 ,
       0.65137815, 0.39720258, 0.78873014, 0.31683612, 0.56809866,
       0.8691274 , 0.4361734 , 0.8021476 , 0.14376682, 0.70426095,
       0.7045813 , 0.21879211, 0.92486763, 0.44214076, 0.90931594,
       0.05980922, 0.18428709, 0.04735528, 0.6748809 , 0.59462476,
       0.5333102 , 0.04332406, 0.5614331 , 0.32966843, 0.5029668 ,
       0.11189432, 0.6071937 , 0.5659447 , 0.00676406, 0.6174417 ,
       0.9121229 , 0.7905241 , 0.99208146, 0.95880175, 0.7919641 ,
       0.28525096, 0.62491673, 0.4780938 , 0.19567518], dtype=float32)

In [11]:
xb[:1][0]

array([[0.19151945, 0.62210876, 0.43772775, 0.7853586 , 0.77997583,
        0.2725926 , 0.27646425, 0.8018722 , 0.95813936, 0.87593263,
        0.35781726, 0.5009951 , 0.6834629 , 0.71270204, 0.37025076,
        0.5611962 , 0.50308317, 0.01376845, 0.7728266 , 0.8826412 ,
        0.364886  , 0.6153962 , 0.07538124, 0.368824  , 0.9331401 ,
        0.65137815, 0.39720258, 0.78873014, 0.31683612, 0.56809866,
        0.8691274 , 0.4361734 , 0.8021476 , 0.14376682, 0.70426095,
        0.7045813 , 0.21879211, 0.92486763, 0.44214076, 0.90931594,
        0.05980922, 0.18428709, 0.04735528, 0.6748809 , 0.59462476,
        0.5333102 , 0.04332406, 0.5614331 , 0.32966843, 0.5029668 ,
        0.11189432, 0.6071937 , 0.5659447 , 0.00676406, 0.6174417 ,
        0.9121229 , 0.7905241 , 0.99208146, 0.95880175, 0.7919641 ,
        0.28525096, 0.62491673, 0.4780938 , 0.19567518]], dtype=float32)

In [10]:
index.reconstruct(0).reshape(1, -1)

array([[0.19151945, 0.62210876, 0.43772775, 0.7853586 , 0.77997583,
        0.2725926 , 0.27646425, 0.8018722 , 0.95813936, 0.87593263,
        0.35781726, 0.5009951 , 0.6834629 , 0.71270204, 0.37025076,
        0.5611962 , 0.50308317, 0.01376845, 0.7728266 , 0.8826412 ,
        0.364886  , 0.6153962 , 0.07538124, 0.368824  , 0.9331401 ,
        0.65137815, 0.39720258, 0.78873014, 0.31683612, 0.56809866,
        0.8691274 , 0.4361734 , 0.8021476 , 0.14376682, 0.70426095,
        0.7045813 , 0.21879211, 0.92486763, 0.44214076, 0.90931594,
        0.05980922, 0.18428709, 0.04735528, 0.6748809 , 0.59462476,
        0.5333102 , 0.04332406, 0.5614331 , 0.32966843, 0.5029668 ,
        0.11189432, 0.6071937 , 0.5659447 , 0.00676406, 0.6174417 ,
        0.9121229 , 0.7905241 , 0.99208146, 0.95880175, 0.7919641 ,
        0.28525096, 0.62491673, 0.4780938 , 0.19567518]], dtype=float32)

> 说明：
> 
> - 将向量 xb 添加到 index。大多数索引除了要添加训练数据外，还需要`index.train`训练来分析向量集的分布。但`IndexFlatL2`可以跳过这个操作。

### 用k-近邻搜索

- 用测试数据来测试

```python
k = 4   # we want to see 4 nearest neighbors
D, I = index.search(xq, k)     # actual search
```
> - xq is the query data
> - k is the num of neigbors you want to search
> - D is the distance matrix between xq and k neigbors
> - I is the index matrix of k neigbors

In [5]:
k = 4                               # k邻近搜索 k=4 
D, I = index.search(xb[:5], k)      # 用xb前五个数据测试
print(I)
print(D)

[[  0 393 363  78]
 [  1 555 277 364]
 [  2 304 101  13]
 [  3 173  18 182]
 [  4 288 370 531]]
[[0.        7.1751733 7.207629  7.2511625]
 [0.        6.3235645 6.684581  6.7999454]
 [0.        5.7964087 6.391736  7.2815123]
 [0.        7.2779055 7.5279865 7.6628466]
 [0.        6.7638035 7.2951202 7.3688145]]


In [6]:
k = 10                               # k邻近搜索 k=10
D, I = index.search(xb[-5:], k)      # 用xb前五个数据测试
print(I)

[[99995 99822 99864 99692 99810 99608 98764 98978 99653 99815]
 [99996 99836 99784 99941 99800 99995 99647 99506 99656 99859]
 [99997 99339 99946 99273 99647 99608 99874 99138 99742 99434]
 [99998 99711 99798 99584 99817 99508 99974 99206 98991 99332]
 [99999 99404 99659 99941 99693 99931 99827 99562 99568 99653]]
[[0.        5.665085  5.704484  5.785391  5.862203  6.327144  6.331679
  6.382374  6.421702  6.436242 ]
 [0.        6.68367   6.842343  6.920699  7.0573673 7.116924  7.1207876
  7.1694236 7.1842127 7.2120733]
 [0.        6.7744017 6.818219  6.88326   6.965914  6.992526  7.3095956
  7.3889666 7.4510303 7.602867 ]
 [0.        7.5281425 7.7377906 7.9008036 7.973149  8.318675  8.52397
  8.649483  8.698359  8.7357855]
 [0.        6.173254  7.0847716 7.2752233 7.599222  7.824944  7.891345
  7.9095006 8.020744  8.248194 ]]


In [14]:
k = 10                               # k邻近搜索 k=10
D, I = index.search(index.reconstruct(1).reshape(1, -1), k)      # 用xb前五个数据测试
print(I)
print(D)

[[   1  555  277  364  617  175 1063  756   77  191]]
[[0.        6.3235645 6.684581  6.7999454 6.8844795 6.919898  6.927656
  7.493     7.4973536 7.5480022]]


距离此向量最近的确实是他本身。

- 用真实数据测试

In [7]:
D, I = index.search(xq, k)          # 执行搜索
print(I[:5])                        # 最初五次查询的结果
print(I[-5:])                       # 最后五次查询的结果

[[ 381  207  210  477  588  329  417  555   93  111]
 [ 526  911  142   72  300  576   64  846  144 1273]
 [ 838  527 1290  425  281  637 1541  197  789  614]
 [ 196  184  164  359  466  599  281 1130  758  427]
 [ 526  377  120  425  545  917  161  416   64  434]]
[[ 9900 10500  9309  9831  9810 10808 10568 10048  9649  9632]
 [11055 10895 10812 11321 10260 10403  9829  9014 10428 10240]
 [11353 11103 10164  9787 10719  9380 10118 11013 10089 10824]
 [10571 10664 10632  9638  9589 10203  9894 10418 11268 11348]
 [ 9628  9554 10036  9582 10304  9622 10103 10016  9229  8943]]


### 完整代码
```python
import numpy as np

d = 64                              # 向量维度
nb = 100000                         # 向量集大小
nq = 10000                          # 查询次数
np.random.seed(1234)                # 结果可复现
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.   # 修正项
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

import faiss
index = faiss.IndexFlatL2(d)        # 构建FlatL2索引

print(index.is_trained)
index.add(xb)                       # 向索引中添加向量
print(index.ntotal)

k = 4                               # k=4的 k邻近搜索
D, I = index.search(xq, k)          # 执行搜索
print(I[:5])                        # 最初五次查询的结果
print(I[-5:])                       # 最后五次查询的结果
```

## 参考

- [官网文档](https://github.com/facebookresearch/faiss/wiki)
- [Faiss-indexes](https://github.com/facebookresearch/faiss/wiki/Faiss-indexes)
- [similarity search](https://github.com/facebookresearch/faiss/wiki)
- [Guidelines to choose an index](https://github.com/facebookresearch/faiss/wiki)
- [Benchmarking scripts](https://github.com/facebookresearch/faiss/blob/master/benchs/README.md)